# AI Agent with Bitcoin Price Fetching and Python Execution Tools

Poetry exercise

Run the following commands every time you want to run a code book.
!pip install poetry
!poetry install
!poetry shell

## Overview

This notebook demonstrates how to create a **LangChain AI Agent** that can dynamically choose between multiple tools to execute user requests. The agent can:
- Fetch the current Bitcoin price from a public API.
- Execute Python code provided by the user in a secure environment.

The AI model powering the agent is `ChatOpenAI`, and we use the `AgentType.OPENAI_MULTI_FUNCTIONS` configuration, which allows the agent to choose the appropriate tool based on user input.

In [1]:
!pip install langchain openai requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 77.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


## Step 1: Import Libraries

In [ ]:
import requests
import subprocess
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

## Step 2: Define Custom Tools

LangChain allows us to define tools that the agent can use. We define two tools:
1. `get_bitcoin_price()`: Fetches the current Bitcoin price in USD.
2. `run_python_code(code)`: Executes a Python script and returns the output.

These tools enable the AI agent to perform dynamic operations based on user queries.

the simplest way to create a tool is to use the @tool decorator and a descriptive docstring

In [ ]:
@tool
def get_bitcoin_price() -> str:
    """
    Fetches the current Bitcoin price in USD from a public cryptocurrency API.
    """
    url = "https://api.coindesk.com/v1/bpi/currentprice/BTC.json"
    response = requests.get(url)
    data = response.json()
    return f"The current Bitcoin price is ${data['bpi']['USD']['rate']} USD."

@tool
def run_python_code(code: str) -> str:
    """
    Executes the given Python code and returns the output.
    """
    try:
        result = subprocess.run(["python3", "-c", code], capture_output=True, text=True, timeout=5)
        return result.stdout if result.stdout else result.stderr
    except Exception as e:
        return str(e)

## Step 3: Initialize AI Agent

We initialize the AI agent using `AgentType.OPENAI_MULTI_FUNCTIONS`, allowing it to decide dynamically which tool to use.

By setting `verbose=True`, we can see how the agent makes decisions in real time.

In [ ]:
# initialize the LLM
llm = ChatOpenAI(temperature=0.7)
# declare the agent
agent = initialize_agent(
    tools=[get_bitcoin_price, run_python_code],
    llm=llm,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    verbose=True
)

## Step 4: Example Usage

In [ ]:
print(agent.run("What is the current Bitcoin price?"))
print(agent.run("Execute this Python code: print(5 * 10)"))

## Exercises for Students

1. **Enhance the Bitcoin Price Tool:** Modify `get_bitcoin_price` to fetch prices in different currencies (e.g., EUR, GBP).
2. **Add Error Handling:** Ensure `run_python_code` handles syntax errors gracefully and returns informative messages. (hard)
3. **Expand the Agent's Capabilities:** Add a new tool that fetches Ethereum prices and let the agent decide when 
